In [51]:
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, f1_score, roc_auc_score, roc_curve, precision_recall_curve, auc, recall_score, confusion_matrix
import sys 
sys.path.append('/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/')
from RF_atomver import *
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler, NearMiss, TomekLinks, ClusterCentroids
from imblearn.ensemble import EasyEnsembleClassifier, BalancedBaggingClassifier
from imblearn.pipeline import Pipeline
from sklearn.metrics import precision_score, recall_score, roc_auc_score, matthews_corrcoef, balanced_accuracy_score, confusion_matrix
from sklearn.datasets import make_classification

In [52]:
# GP: GP matern 
# RF: just RF, not Grid search 
# let's remove literautre sets 
# call this file: final_paper_results (keep smote and adasyn) 

In [53]:
# GP matern
matern_path = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/GP/GP_matern_kernel/'
gp_matern = pd.read_csv(matern_path+'GP_maternkern_test_results.csv')
gp_matern.columns

Index(['model', 'NEK', 'strategy', 'feat_type', 'cm', 'prediction_type',
       'recall', 'ROC-AUC', 'MCC', 'Balanced Accuracy', 'f1', 'accuracy',
       'precision', 'specificity', 'TN', 'FN', 'FP', 'TP'],
      dtype='object')

In [54]:
# RF 
RF_path= '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/RF/RF_results/'
og_RF = pd.read_csv(RF_path+'RF_test_results_all_NEK.csv')
gp_matern['set'] = 'original' 
og_RF['set'] = 'original' 
gp_matern['modeling_type'] = 'GP_matern' 
gp_matern['RF_type'] = 'N/A' 
og_RF['modeling_type'] = 'RF' 

# cross validation folds 
validation = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/undersampler_validation/idea5_dir/'
folds = pd.read_csv(f'{validation}RF_results_all_folds.csv')
folds['set'] = folds['train_fold'] 
folds=folds.drop(columns=['train_fold'])
folds['model']=folds['NEK'].astype(str)+'_'+folds['feat_type']+'_'+folds['strategy']+'_'+folds['RF_type'] 

# 2nd undersample inactive training batch
batch2 = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/undersampler_validation/under_batch2_jp/'
rf_batch2 = pd.read_csv(f'{batch2}RF_results_UDNER_batch2.csv')
gp_batch2= pd.read_csv(f'{batch2}GP_matern_results_UDNER_batch2.csv')
gp_batch2['RF_type'] = 'N/A' 
gp_batch2['modeling_type'] ='GP_matern'
rf_batch2['modeling_type'] ='RF'
batch2_results = pd.concat([rf_batch2, gp_batch2])
batch2_results['set'] = 'batch2_inactives'
batch2_results

final_paper_model = pd.concat([gp_matern, og_RF, folds, batch2_results])
final_paper_model = final_paper_model.replace('scaled', 'raw')
final_paper_model.to_csv('/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/general_NEK/final_paper_models.csv',index=False)
final_paper_model.to_excel('/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/general_NEK/final_paper_models.xlsx',index=False)

In [56]:
final_paper_model['set'].value_counts()


set
original            240
fold1                84
fold2                84
fold3                84
fold4                84
fold5                84
batch2_inactives     24
Name: count, dtype: int64

In [60]:
smotes = final_paper_model[final_paper_model['strategy'] == 'SMOTE']
smotes

,model,NEK,strategy,feat_type,cm,prediction_type,recall,ROC-AUC,MCC,Balanced Accuracy,...,accuracy,precision,specificity,TN,FN,FP,TP,set,modeling_type,RF_type
2,NEK2_binding_moe_SMOTE_GP_Dirichlet_matern,NEK2_binding,SMOTE,moe,"[270, 1, 9, 3]",TN,0.250000,0.623155,0.420460,0.623155,...,0.964664,0.750000,0.996310,270,9,1,3,original,GP_matern,N/A
6,NEK2_binding_mfp_SMOTE_GP_Dirichlet_matern,NEK2_binding,SMOTE,mfp,"[271, 0, 10, 2]",TN,0.166667,0.583333,0.400918,0.583333,...,0.964664,1.000000,1.000000,271,10,0,2,original,GP_matern,N/A
10,NEK2_inhibition_moe_SMOTE_GP_Dirichlet_matern,NEK2_inhibition,SMOTE,moe,"[377, 3, 12, 16]",TN,0.571429,0.781767,0.676144,0.781767,...,0.963235,0.842105,0.992105,377,12,3,16,original,GP_matern,N/A
14,NEK2_inhibition_mfp_SMOTE_GP_Dirichlet_matern,NEK2_inhibition,SMOTE,mfp,"[378, 2, 13, 15]",TN,0.535714,0.765226,0.671125,0.765226,...,0.963235,0.882353,0.994737,378,13,2,15,original,GP_matern,N/A
18,NEK3_binding_moe_SMOTE_GP_Dirichlet_matern,NEK3_binding,SMOTE,moe,"[259, 6, 15, 2]",TN,0.117647,0.547503,0.136200,0.547503,...,0.925532,0.250000,0.977358,259,15,6,2,original,GP_matern,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
410,NEK9_inhibition_mfp_SMOTE_RF,NEK9_inhibition,SMOTE,mfp,"[66, 5, 4, 5]",TN,0.555556,0.742567,0.463513,0.742567,...,0.887500,0.500000,0.929577,66,4,5,5,fold1,RF,RF
411,NEK9_inhibition_mfp_SMOTE_RF,NEK9_inhibition,SMOTE,mfp,"[65, 5, 2, 7]",TN,0.777778,0.853175,0.625262,0.853175,...,0.911392,0.583333,0.928571,65,2,5,7,fold2,RF,RF
412,NEK9_inhibition_mfp_SMOTE_RF,NEK9_inhibition,SMOTE,mfp,"[68, 2, 4, 4]",TN,0.500000,0.735714,0.536745,0.735714,...,0.923077,0.666667,0.971429,68,4,2,4,fold3,RF,RF
413,NEK9_inhibition_mfp_SMOTE_RF,NEK9_inhibition,SMOTE,mfp,"[68, 2, 2, 6]",FP,0.750000,0.860714,0.721429,0.860714,...,0.948718,0.750000,0.971429,68,2,2,6,fold4,RF,RF
